# Part 1: Pre-processing the files (ETL)

In [25]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import prettytable

### Creating list of filepaths to process original event csv data files

In [4]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [6]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

print(f"Total rows : {len(full_data_rows_list)}")
print(f"Sample data:\n {full_data_rows_list[:5]}")

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
#The register_dialect function creates a new dialect with the specified name
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

Total rows : 8056
Sample data:
 [['', 'Logged In', 'Walter', 'M', '0', 'Frye', '', 'free', 'San Francisco-Oakland-Hayward, CA', 'GET', 'Home', '1.54092E+12', '38', '', '200', '1.54111E+12', '39'], ['', 'Logged In', 'Kaylee', 'F', '0', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Home', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ["Des'ree", 'Logged In', 'Kaylee', 'F', '1', 'Summers', '246.30812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'You Gotta Be', '200', '1.54111E+12', '8'], ['', 'Logged In', 'Kaylee', 'F', '2', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Upgrade', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ['Mr Oizo', 'Logged In', 'Kaylee', 'F', '3', 'Summers', '144.03873', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'Flat 55', '200', '1.54111E+12', '8']]


In [7]:
# checking the number of rows in new event csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra Queries

### Now you are ready to work with the CSV file titled , located within the Workspace directory. 
### The event_datafile_new.csv contains the following columns:

- artist
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

<img src="img.jpg">

### create cluster

In [11]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("Connection Established !!")
except Exception as e:
    print(f"Connection Failed !! Error : {e}")

Connection Established !!


### create keyspace 

In [14]:

keyspace_query = """CREATE KEYSPACE IF NOT EXISTS music_user_history 
                    with REPLICATION = 
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                """

# Creating Keyspace
try:
    session.execute(keyspace_query)
except Exception as e:
    print(f"Failed to create keyspace!! Error : {e}")

### set keyspace

In [15]:
try:
    session.set_keyspace('music_user_history')
except Exception as e:
    print(e)

## Queries to ask the following three questions of the data¶
- #### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
- #### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- #### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### query 1

In [17]:
# Creating table for query1 
create_query1 = """CREATE TABLE IF NOT EXISTS session_item (artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"""

try: 
    session.execute(create_query1)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


In [18]:
# Using the event file
file = 'event_datafile_new.csv'

# Reading csv file and inserting rows into cassandra tables.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item (artist, song, length, sessionId, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s) "
        session.execute(query, (line[0], line[10], float(line[5]), int(line[8]), int(line[3])) )

In [28]:
# SELECT statement to verify the data was entered into the table
select_query1 = "SELECT artist, song, length FROM  session_item where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)
    
t = prettytable.PrettyTable(['Artist', 'Song', 'Length'])
for row in rows:
    t.add_row([row.artist, row.song, row.length])
print(t)

+-----------+------+--------------------+
|   Artist  | Song |       Length       |
+-----------+------+--------------------+
| Faithless |  50  | 495.30731201171875 |
+-----------+------+--------------------+


### query 2

In [21]:
## Query 2: Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userId = 10, sessionId = 182

query = "CREATE TABLE IF NOT EXISTS user_session"
# note: itemInSession added to primary key to order ascending by this value based on query requirements
query = query + "(userId INT , sessionId INT, itemInSession SMALLINT, artist TEXT, song TEXT, user TEXT, PRIMARY KEY ((userid, sessionid), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [23]:

# Insert data into artist_song_user_session_listen_history

file = 'event_datafile_new.csv'

# Loop through CSV file and insert data into Cassandra
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statement 
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song, user)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        ## Try INSERT statement
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + " " + line[4]))
        except Exception as e:
            print(e)

In [27]:
## Query 2: Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userId = 10, sessionId = 182
query = "SELECT artist, song, user FROM user_session WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
t = prettytable.PrettyTable(['Artist', 'Song', 'User'])
for row in rows:
    t.add_row([row.artist, row.song, row.user])
print(t)

+-------------------+------------------------------------------------------+-------------+
|       Artist      |                         Song                         |     User    |
+-------------------+------------------------------------------------------+-------------+
|  Down To The Bone |                  Keep On Keepin' On                  | Sylvie Cruz |
|    Three Drives   |                     Greece 2000                      | Sylvie Cruz |
| Sebastien Tellier |                      Kilometer                       | Sylvie Cruz |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz |
+-------------------+------------------------------------------------------+-------------+


### query 3

In [40]:
## Query 3: Return the user name (first and last) in the music app history for those who listened to the song 'All Hands Against His Own'

# song and userId may be repeated for multiple song listens by same user
# did not use user's name in primary key since multiple users could have same name and have listened to same song

query = "CREATE TABLE IF NOT EXISTS user_song_listen_history"
query = query + "(song TEXT, userId INT, user TEXT, PRIMARY KEY ((song), userId))"
try:
    session.execute(query)
    print("table created")
except Exception as e:
    print(e)

table created


In [41]:
# Insert data into user_song_listen_history

file = 'event_datafile_new.csv'

# Loop through CSV file and insert data into Cassandra
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statement 
        query = "INSERT INTO user_song_listen_history (song, userId, user)"
        query = query + " VALUES (%s, %s, %s)"
        ## Try INSERT statement
        try:
            session.execute(query, (line[9], int(line[10]), line[1] + " " + line[4]))
        except Exception as e:
            print(e)

In [42]:
## Query 3: Return the user name (first and last) in the music app history for those who listened to the song 'All Hands Against His Own'

query = "SELECT user FROM user_song_listen_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
t = prettytable.PrettyTable(['User'])
for row in rows:
    t.add_row([row.user])
print(t)

+------------------+
|       User       |
+------------------+
| Jacqueline Lynch |
|   Tegan Levine   |
|   Sara Johnson   |
+------------------+


### Drop the tables before closing out the sessions

In [ ]:
query = "DROP TABLE IF EXISTS session_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_song_listen_history"
try:
    rows = session.execute(query)
except Exceptio

### Close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()